## Neuro File Transfer (.ps1)

This should be kept and run in the Compute Canada folder where Oddity Moca and Neuropsych scans are backed up and sorted  It will send new files (24hours) to Compute Canada.

*Remember to replace with actual path of your scratch folder*

General

In [ ]:
$currentDate = Get-Date

$username = Read-Host "Please enter your username"

$remotePathBase = "/home/$username/scratch/Actual_Final_Neuropsych_Battery/"
Write-Output "The constructed remote path is: $remotePathBase"
$pathConfirmation = Read-Host "Is this path correct? [Y/N]"

if ($pathConfirmation -ne "Y") {
    $remotePathBase = Read-Host "Please enter the full correct remote path"
}

$recentDirectories = Get-ChildItem -Directory | Where-Object { ($currentDate - $_.CreationTime).TotalHours -le 24 }

$transferredCount = 0
$transferredDirectories = @()

foreach ($dir in $recentDirectories) {
    $remotePath = $remotePathBase + $dir.Name
    $directorySize = (Get-ChildItem $dir.FullName -Recurse | Measure-Object -Property Length -Sum).Sum / 1KB

    $userInput = Read-Host "Do you want to push $($dir.Name) [Size: $($directorySize) KB] to the server at `"$remotePath`"? [Y/N]"

    if ($userInput -eq "Y") {

        scp -r $dir.FullName "$username@graham.computecanada.ca:`"$remotePath`""
        if ($?) {
            $transferredCount++
            $transferredDirectories += $dir.Name
            Write-Output ("Directory '{0}' transferred. Size: {1:N2} MB" -f $dir.Name, $directorySize)
        } else {
            Write-Warning "Failed to transfer directory: $dir"
        }
    }
}

Write-Output "Transfer process complete."
Write-Output "$transferredCount directory(ies) were transferred:"
$transferredDirectories | ForEach-Object {
    Write-Output $_
}

Read-Host -Prompt "Press Enter to Close"


NeuropsychBattery_Auto_Sync.ps1

In [ ]:
$currentDate = Get-Date

$recentDirectories = Get-ChildItem -Directory | Where-Object { ($currentDate - $_.CreationTime).TotalHours -le 24 }

$transferredCount = 0
$transferredDirectories = @()

$remotePathBase = "/home/samhsu03/scratch/Actual_Final_Neuropsych_Battery/"

foreach ($dir in $recentDirectories) {
    $remotePath = $remotePathBase + $dir.Name

    scp -r $dir.FullName "samhsu03@graham.computecanada.ca:`"$remotePath`""
    if ($?) {
        $transferredCount++
        $transferredDirectories += $dir.Name
        
        $directorySize = (Get-ChildItem $dir.FullName -Recurse | Measure-Object -Property Length -Sum).Sum / 1MB
        Write-Output ("Directory '{0}' transferred. Size: {1:N2} MB" -f $dir.Name, $directorySize)
    } else {
        Write-Warning "Failed to transfer directory: $dir"
    }
}

Write-Output "Transfer process complete."
Write-Output "$transferredCount directory(ies) were transferred:"
$transferredDirectories | ForEach-Object {
    Write-Output $_
}